In [1]:
import h5py
import torch

In [2]:
# 函数：向嵌套字典添加新值
def add_to_nested_dict(nested_dict, k1, k2, value):
    # 如果 k1 不存在，创建新的子字典
    if k1 not in nested_dict:
        nested_dict[k1] = {}
    
    # 如果 k2 不存在，创建新的列表
    if k2 not in nested_dict[k1]:
        nested_dict[k1][k2] = []
    
    # 将 value 追加到 k2 对应的列表
    nested_dict[k1][k2].append(value)

In [3]:
from datetime import datetime
all_content = {}
output_file = "hidden_states.h5"
with h5py.File(output_file, "r") as f_read:
    for k in f_read.keys():
        #print("文件中的键：", list(f_read[k]))
        for ds in list(f_read[k]):

            ts = datetime.strptime(ds.split("_")[-1], "%Y-%m-%d %H:%M:%S%z")
            day = str(ts.date())
            #print(day)
            # {k: {ts: f_read[k][ds][:]}}
            # all_content[k] = {ts.date():f_read[k][ds][:]}
            add_to_nested_dict(all_content, k, day, torch.tensor(f_read[k][ds][:]).to(torch.bfloat16))
        #data = f_read[k]['hidden_states_2005-10-13 19:00:40+00:00'][:]
        #break

In [4]:
def swap_hierarchy(A):
    """
    交换字典 A 的层级，从 {name: {timestamps: [value, value]}} 转换为
    {timestamps: {name: [value, value]}}.
    
    参数:
        A: 输入字典，结构为 {name: {timestamps: [value, value]}}
    
    返回:
        new_A: 转换后的字典，结构为 {timestamps: {name: [value, value]}}
    """
    new_A = {}
    
    # 遍历原始字典
    for name, timestamp_dict in A.items():
        for timestamp, values in timestamp_dict.items():
            # 如果 timestamp 不在 new_A 中，初始化
            if timestamp not in new_A:
                new_A[timestamp] = {}
            # 将 name 和 values 添加到对应的 timestamp 下
            new_A[timestamp][name] = values
    
    return new_A

# 示例字典
A = {
    "name1": {"2023-01-01": [10, 20], "2023-06-01": [30, 40]},
    "name2": {"2023-01-01": [5, 15], "2023-07-01": [25, 35]}
}

# 转换层级
result = swap_hierarchy(A)
print(result)

{'2023-01-01': {'name1': [10, 20], 'name2': [5, 15]}, '2023-06-01': {'name1': [30, 40]}, '2023-07-01': {'name2': [25, 35]}}


In [5]:
def merge_values(A):
    """
    将字典 A 的结构从 {timestamps: {name: [value, value]}} 转换为
    {timestamps: [value, value]}，合并每个 timestamps 下所有 name 的 value。
    
    参数:
        A: 输入字典，结构为 {timestamps: {name: [value, value]}}
    
    返回:
        new_A: 转换后的字典，结构为 {timestamps: [value, value]}
    """
    new_A = {}
    
    # 遍历原始字典
    for timestamp, name_dict in A.items():
        # 合并所有 name 的 value 列表
        merged_values = []
        for values in name_dict.values():
            merged_values.extend(values)
        # 存储到新字典
        new_A[timestamp] = merged_values
    
    return new_A

# 示例字典
A = {
    "2023-01-01": {"name1": [10, 20], "name2": [5, 15]},
    "2023-06-01": {"name1": [30, 40]},
    "2023-07-01": {"name2": [25, 35]}
}

# 合并 value
result = merge_values(A)
print(result)

{'2023-01-01': [10, 20, 5, 15], '2023-06-01': [30, 40], '2023-07-01': [25, 35]}


In [12]:
import torch

def mean_pool_values(A):
    """
    将字典 A 的结构从 {timestamps: [value, value]} 转换为 {timestamps: value}，
    其中 value 是 torch.Tensor，通过平均池化合并每个 timestamps 下的 value 列表。
    
    参数:
        A: 输入字典，结构为 {timestamps: [torch.Tensor, torch.Tensor]}
    
    返回:
        new_A: 转换后的字典，结构为 {timestamps: torch.Tensor}
    """
    new_A = {}
    
    # 遍历原始字典
    for timestamp, value_list in A.items():
        # 将 value 列表堆叠为张量，形状为 (num_values, ...)
        stacked_values = torch.stack(value_list)
        # 沿第0维（num_values）取平均，得到单个张量
        pooled_value = torch.mean(stacked_values, dim=0)
        # 存储到新字典
        new_A[timestamp] = torch.mean(pooled_value, dim=0)
    
    return new_A

# 示例字典
A = {
    "2023-01-01": [torch.tensor([1.0, 2.0]), torch.tensor([3.0, 4.0])],
    "2023-06-01": [torch.tensor([5.0, 6.0]), torch.tensor([7.0, 8.0])],
    "2023-07-01": [torch.tensor([9.0, 10.0])]
}

# 平均池化
result = mean_pool_values(A)
for timestamp, value in result.items():
    print(f"{timestamp}: {value}")

2023-01-01: 2.5
2023-06-01: 6.5
2023-07-01: 9.5


In [13]:
mean_content = mean_pool_values(merge_values(swap_hierarchy(all_content)))

In [17]:
from datetime import datetime

def get_values_before_date(A, cutoff_date_str, date_format="%Y-%m-%d"):
    """
    返回字典 A 中所有 timestamps 小于 cutoff_date 的 value 列表。
    
    参数:
        A: 输入字典，结构为 {timestamps: value}
        cutoff_date_str: 截止日期字符串（如 "2023-06-01"）
        date_format: 日期格式，默认为 "%Y-%m-%d"
    
    返回:
        values: 小于截止日期的所有 value 列表
    """
    # 将截止日期转换为 datetime 对象
    cutoff_date = datetime.strptime(cutoff_date_str, date_format)
    
    # 初始化结果列表
    values = []
    
    # 遍历字典
    for timestamp, value in A.items():
        # 将 timestamp 转换为 datetime 对象
        timestamp_date = datetime.strptime(timestamp, date_format)
        # 如果 timestamp 小于截止日期，添加对应的 value
        if timestamp_date < cutoff_date:
            values.append(value)
    
    return values

# 示例字典
import torch
A = {
    "2023-01-01": torch.tensor([1.0, 2.0]),
    "2023-06-01": torch.tensor([3.0, 4.0]),
    "2023-07-01": torch.tensor([5.0, 6.0]),
    "2024-01-01": torch.tensor([7.0, 8.0])
}

# 示例调用
cutoff_date = "2023-07-01"
result = get_values_before_date(A, cutoff_date)
print(f"Values before {cutoff_date}: {result}")

Values before 2023-07-01: [tensor([1., 2.]), tensor([3., 4.])]


In [24]:
len(get_values_before_date(mean_content, "2017-07-01")[-200:])

200

In [26]:
from datetime import datetime

def is_timestamp(obj):
    """
    判断变量是否是 datetime.datetime 对象。
    
    参数:
        obj: 要检查的变量
    
    返回:
        bool: 如果是 datetime.datetime 对象，返回 True，否则返回 False
    """
    return isinstance(obj, datetime)

# 示例测试
# 创建一些测试变量
var1 = datetime(2023, 1, 1)  # datetime 对象
var2 = "2023-01-01"          # 字符串
var3 = 1697059200            # Unix 时间戳（整数）
var4 = None                  # None 值

# 测试
print(is_timestamp(var1))  # True
print(is_timestamp(var2))  # False
print(is_timestamp(var3))  # False
print(is_timestamp(var4))  # False

True


KeyboardInterrupt: 